In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import re
from PIL import Image
from skimage.transform import resize
from sklearn.model_selection import train_test_split

In [2]:
import keras
from keras.models import Model, Input, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K


Using TensorFlow backend.


In [48]:
## get all the array values from the npy files 
train_img = np.load('train_images.npy')
train_mask = np.load('train_masks.npy')

In [49]:
########need to resize the images to pass it to U-net 
import random
indices = np.random.choice(range(len(train_img)), replace = False ,size = 100)

X = train_img[indices, :,:,np.newaxis]/255
y= train_mask[indices,:,:,np.newaxis]/255

In [50]:
# X = X[:,:,:,np.newaxis] / 255
# y = y[:,:,:,np.newaxis] / 255
print("X shape : ", X.shape)
print("y shape : ", y.shape)

X shape :  (100, 128, 128, 1)
y shape :  (100, 128, 128, 1)


In [8]:
def dice_coefficent(img, mask):
    smooth =1
    img_points = K.flatten(img)
    mask_points =K.flatten(mask)
    intersection = K.sum(img_points*mask_points)
    
    coef = (2.*intersection + smooth) / (K.sum(img_points)+K.sum(mask_points) +smooth)
    return coef

def loss_dice(img,mask):
    loss = - dice_coefficent(img,mask)
    return loss
    

In [9]:
IMG_HEIGHT =128
IMG_WIDTH = 128

inputs = Input((IMG_HEIGHT, IMG_WIDTH,1))

conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])
model.compile(optimizer=Adam(lr = 1e-5), loss=loss_dice, metrics=[dice_coefficent])

In [12]:
results = model.fit(X, y, validation_split=0.2,batch_size=4, epochs=20,shuffle=True)

Train on 80 samples, validate on 20 samples
Epoch 1/20
80/80 [==============================] - 17s 214ms/step - loss: -0.0240 - dice_coefficent: 0.0240 - val_loss: -0.0236 - val_dice_coefficent: 0.0236
Epoch 2/20
80/80 [==============================] - 17s 218ms/step - loss: -0.0241 - dice_coefficent: 0.0241 - val_loss: -0.0236 - val_dice_coefficent: 0.0236
Epoch 3/20
80/80 [==============================] - 17s 218ms/step - loss: -0.0242 - dice_coefficent: 0.0242 - val_loss: -0.0237 - val_dice_coefficent: 0.0237
Epoch 4/20
80/80 [==============================] - 17s 215ms/step - loss: -0.0243 - dice_coefficent: 0.0243 - val_loss: -0.0239 - val_dice_coefficent: 0.0239
Epoch 5/20
80/80 [==============================] - 17s 210ms/step - loss: -0.0245 - dice_coefficent: 0.0245 - val_loss: -0.0242 - val_dice_coefficent: 0.0242
Epoch 6/20
80/80 [==============================] - 17s 216ms/step - loss: -0.0249 - dice_coefficent: 0.0249 - val_loss: -0.0246 - val_dice_coefficent: 0.0246
Ep

In [54]:
def Generator(X_list, y_list, batch_size = 16):
    
    c = 0
    for i in range(c,c+batch_size):
        X_list[i - c] = X_list[i]
        y_list[i - c] = y_list[i]
        
    c += batch_size
    if(c+batch_size >= len(X_list)):
        c = 0
    yield X, y   

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.3, random_state = 1)

epochs = 10
batch_size = 8
steps_per_epoch = int(len(X_train) / batch_size)
validation_steps = int(len(X_val) / batch_size)

train_gen = Generator(X_train, y_train, batch_size = batch_size)
val_gen = Generator(X_val, y_val, batch_size = batch_size)

model = Model(inputs=[inputs], outputs=[conv10])
model.compile(optimizer=Adam(lr = 1e-5), loss=loss_dice, metrics=[dice_coefficent])
history = model.fit_generator(train_gen, steps_per_epoch=steps_per_epoch, epochs = epochs,
                             validation_data = val_gen, validation_steps = validation_steps)

Epoch 1/10
1/8 [==>...........................] - ETA: 2:07 - loss: -0.1339 - dice_coefficent: 0.1339

StopIteration: 

In [55]:
test_list = os.listdir("input/test/test")
reg =re.compile("[0-9]+")

temp1 = list(map(lambda x: reg.match(x).group(), test_list)) 
temp1 = list(map(int, temp1))
test_list = [x for _,x in sorted(zip(temp1, test_list))]
X=[]
for i in range(len(test_list)):
    img = Image.open('input/test/test/'+test_list[i])
    img_resized = img.resize((128,128))
    X.append(np.array(img_resized))
    
X= np.asarray(X)
X= X[:,:,:,np.newaxis]/255

np.save('xtest.npy',X)

In [39]:
y_pred =model.predict(X)

In [45]:
def run_length_enc(label):
    from itertools import chain
    x = label.transpose().flatten()
    y = np.where(x > 0)[0]
    if len(y) < 10:  # consider as empty
        return ''
    z = np.where(np.diff(y) > 1)[0]
    start = np.insert(y[z+1], 0, y[0])
    end = np.append(y[z], y[-1])
    length = end - start
    res = [[s+1, l+1] for s, l in zip(list(start), list(length))]
    res = list(chain.from_iterable(res))
    return ' '.join([str(r) for r in res])


rles = []
for i in range(X.shape[0]):
    img = y_pred[i, :, :, 0]
    img = img > 0.5
    img = resize(img, (420, 580), preserve_range=True)
    rle = run_length_enc(img)
    rles.append(rle)
    if i % 100 == 0:
        print('{}/{}'.format(i, X.shape[0]), end = "\r")
         
sub = pd.read_csv('input/sample_submission.csv')
sub['pixels'] = rles
sub.to_csv("submission.csv", index = False)

C:\Users\Sailalitha\Anaconda3\envs\myenv\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
